## *Clubhoue is an invite-only audio chat social network starting with preliminary release for iOS only*


In [ ]:
"""
Author          : Rohan Madhale
Date created on : May 2021
Title           : EDA for Clubhouse data
"""

print('')

## Summary
* The dataset has *1300515* unique user ids and **1300514** unique users.
* Each user has following details :
    1. user_id
    2. name
    3. photo_url
    4. username
    5. twitter
    6. instagram
    7. num_followers
    8. num_following
    9. time_created
    10. invited_by_user_profile
* Columns with missing values - *photo_url, username, twitter, instagram, invited_by_user_profile*
* 1 username had 2 unique user_id's
* **Jared Leto, Tiffany Haddish & Van Jones** are the top followed public figures
* **December 2020** has the highest user registration 
* The user arer divided into **3** clusters : **Less than 1k followers, Between 1k & 10k followers & More than 10k followers**
* **41%** of users with **Less than 1k followers** have not linked Instagram and Twitter to their profile
* For users with **More than 1k followers** more than **50%** have linked both Instagram and Twitter to their profile
* Among all 3 groups more users have linked **only Instagram** as compared to **only twitter**
* Only **5%** of the profiles have profile pictures
* **3214** have no invite linked to their profile
* **David Castain** has invited the most nummber of users
* For Users with **More than 1k followers, 56%** of the users have a complete profile
* While **80%** of users with **Less than 1k followers** are missing some information from their profile
* Emoji based observations
   1. **213** user have atleast 1 emoji in their username
   2. *smiling face with tear* is the most common emoji 
   3. **Four** users have four different flag emoji(United States, Sweden, Canada and Belize) in their username 
   4. **17** users have used **six** different skin toned emojis
   5. **Six** Users have used *Rocket emoji*; This emoji implies that either the user works with social media growth related 
      metrics or startup or investor in Dogecoin
   6. One user is artist as the user has artist palette in their username (linked instagram confirms this)
   7. **Three** users have *camera emoji* in their username. Exactly one user works with filmaker/video content creator 
   8. **Three** users work in tech
   9. **Five** users are female and **three** users are male

### A. Import necessary libraries

In [ ]:
###

import pandas as pd
import numpy as np
import sqlite3
from tabulate import tabulate
import plotly.graph_objects as go
import plotly.figure_factory as ff
import chart_studio.plotly as py
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
from plotly.subplots import make_subplots
from advertools import extract_emoji

###

### B. Reading data into pandas dataframe

In [ ]:
### 
#Read Clubhouse_Dataset_v1.db file into pandas dataframe

cnx = sqlite3.connect('Clubhouse_Dataset_v1.db')
user_data = pd.read_sql_query("SELECT * from user", cnx)

###

### C. Data cleaning and formatting

In [ ]:
# C1. Get column names and check data type of columns

###

# Get column names and check data type of columns
print('Columns                   data_type')
print(len('Columns                   data_type')*'-')
print(user_data.dtypes.to_string())

###

In [ ]:
# C2. Check columns for null values

###

print('Columns with null entries')
print(len('Columns with null entries')*'-')
for col in list(user_data.columns):
    if (user_data[user_data[col] == 'null'].shape[0] > 0) | (user_data[user_data[col].isna()].shape[0] > 0):
        print(col)
    else:
        pass

###

In [ ]:
# C3. Check unique entries

###

# Check number of uninque rows in the dataset
if len(user_data.user_id.unique()) == len(user_data):
    print('All user_ids are unique')
else:
    print('Duplicate user_ids present')

###

In [ ]:
# C4. Check if same username duplicate is present

###

print('Duplicate username:')
print(user_data[user_data.duplicated(['username'], keep=False)]
      [['user_id', 'name', 'username']].to_string(index=False))

# Drop user_id = 39784 as has lowest number of followers and follows zero accounts
user_data = user_data[user_data['user_id'] != 39784]

###

In [ ]:
# C5. Data cleaning & Formatting

###

# Replace None with 0
user_data.fillna(0, inplace=True)

# Replace 'null' which are present as string 
user_data.replace('null',0, inplace=True)
user_data['name'].replace('','none', inplace=True)
user_data.replace('', 0, inplace=True)

# Convert data to Year-Month format
user_data['time_created'] = pd.to_datetime(user_data['time_created'], format='%Y-%m').dt.strftime('%Y-%m')

###

### D. EDA 

In [ ]:
# D1. Get total number of users

###

print(f'Total number of user : {len(user_data.user_id.unique())}')

###

In [ ]:
# D2. Get Top followed users

# print('Top 10 users with most followers:')
###

print(user_data.sort_values(["num_followers"], ascending=[False]).head(
    10)[["name", 'num_followers']].to_string(index=False))

print('\nJared Leto, Tiffany Haddish & Van Jones are the top followed public figures')
###

In [ ]:
# D3. Check number of users present in each month to guage app growth

###

x = pd.DataFrame(user_data.time_created.value_counts()).sort_index()

# Scatter plot 
fig = px.scatter(x, x=x.index, y='time_created', hover_data=['time_created'], template='simple_white',
                 labels={
                     "x": "Month",
                     "time_created": "Number of Users"},
                 title="Number of users in each month")

# Add line trace to the scatter plot 
fig.add_trace(go.Scatter(x=x.index, y=x.time_created, mode='lines'))

# Add titles and change font
fig.update_layout(
#     title="Number of users in each month",
    xaxis_title="Month",
    yaxis_title="Numer of Users",
    legend_title="",
    showlegend=False,
    font=dict(
        family="Raleway, monospace",
        size=12,
        color="RebeccaPurple"
    ),
    title={
        'text': "Number of users in each month",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
)


# Set X-axis to 1 Month interval
fig.update_xaxes(
    dtick="M1")

fig.show()

###

In [ ]:
print('November and December 2020 saw a sharp increase in number of new users joining the platform ')

In [ ]:
# D4. Divide users into 3 groups based in number of followers
# 1st Group < 1k followers
# 2nd Group >= 1k & <= 10k followers
# 3rd Group > 10k followers

###

user_data["follower_status"] = np.where(
    user_data["num_followers"] < 1000,
    "Less than 1k followers",
    0,
)

user_data["follower_status"] = np.where(
    (user_data["num_followers"] >= 1000) & (user_data["num_followers"] <= 10000),
    "Between 1k & 10k followers",
    user_data["follower_status"],
)

user_data["follower_status"] = np.where(
    user_data["num_followers"] > 10000, "More than 10k followers",
    user_data["follower_status"],
)


print('Follwers status :')
# print(user_data.groupby(['follower_status']).size().sort_values(ascending=False).to_string())
print((pd.DataFrame(user_data.follower_status.value_counts())).rename(columns={'follower_status':'Count'}))

###

In [ ]:
# D5. Create column based on social media profile linked to clubhouse profile

###

user_data['social_media'] = np.where(
    (user_data['instagram'] == 0) & (user_data['twitter'] != 0), 'Only Twitter', 0)
user_data['social_media'] = np.where(
    (user_data['instagram'] != 0) & (user_data['twitter'] == 0), 'Only Instagram', user_data['social_media'])
user_data['social_media'] = np.where(
    (user_data['instagram'] != 0) & (user_data['twitter'] != 0), 'Both Instagram & Twitter', user_data['social_media'])
user_data['social_media'] = np.where(
    (user_data['instagram'] == 0) & (user_data['twitter'] == 0), 'None', user_data['social_media'])

###

In [ ]:
# D6. Check status of other social media profiles are linked to Clubhouse profile based on number of followers

###

tx = user_data.groupby(['social_media','follower_status']).size().reset_index()
fig = make_subplots(
    rows=1,
    cols=3,
    specs=[[{"type": "domain"}, {"type": "domain"}, {"type": "domain"}]],
    subplot_titles=[
        "Profiles with<br>Less than 1k followers",
        "Profiles with<br>Between 1k & 10k followers",
        "Profiles with<br>More than 10k followers",
    ],
)
fig.add_trace(go.Pie(labels=tx[tx['follower_status']=='Less than 1k followers'].social_media, 
                     values=tx[tx['follower_status']=='Less than 1k followers'][0], name=''), 1, 1)

fig.add_trace(go.Pie(labels=tx[tx['follower_status']=='Between 1k & 10k followers'].social_media, 
                     values=tx[tx['follower_status']=='Between 1k & 10k followers'][0], name=''), 1, 2)

fig.add_trace(go.Pie(labels=tx[tx['follower_status']=='More than 10k followers'].social_media, 
                     values=tx[tx['follower_status']=='More than 10k followers'][0], name=''), 1, 3)

colors = ['lightcyan', 'cyan', 'royalblue', 'darkblue']
fig.update_traces(marker=dict(colors=colors))

fig.update_layout(
    legend_title="Social Media",
    font=dict(family="Raleway, monospace", size=12, color="RebeccaPurple"),
    title={
        "text": "Distribution of other social media profiles linked to Clubhouse profile",
        "y": 0.96,
        "x": 0.39,
        "xanchor": "center",
        "yanchor": "top",
    },
)
fig.update_layout(
    legend=dict(x=1.1,y=0.7)
)


fig.show()

###

In [ ]:
print('41% of Users with less than 1k followers have not linked Instagram and Twitter to their profile')
print('For users with more than 1k followers more than 50% have linked both Instagram and Twitter to their profile')
print('Among all 3 groups more users have linked only Instagram as compared to only twitter')

In [ ]:
# D7. Check if profile picture is present linked to the profile

###

user_data['profile_photo_status'] = np.where(
    user_data['photo_url'] == 0, 'Yes', 'No')
fig = px.histogram(user_data, x="profile_photo_status", labels={
                   "profile_photo_status": "Profile Picture Status", }, color_discrete_sequence=['cyan'])
fig.update_layout(
    xaxis_title="Profile Picture Status",
    yaxis_title="Numer of Users",
    legend_title="",
    showlegend=False,
    font=dict(
        family="Raleway, monospace",
        size=12,
        color="RebeccaPurple"
    ),
    title={
        'text': "Histogram of Profile Picture Status",
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    template='simple_white'
)
fig.show()

###

In [ ]:
print('Only 5% of the profiles have profile pictures')

In [ ]:
# D8. Check for top inviters

###

invite = user_data.groupby(['invited_by_user_profile']).size().reset_index().sort_values([0], ascending=[False])

invite['invited_by_user_profile'].replace(0, 'No Inviter', inplace=True)


df1 = invite[invite['invited_by_user_profile'] == 'No Inviter']
df2 = pd.merge(invite, user_data[['user_id', 'name']], left_on='invited_by_user_profile',
               right_on='user_id').sort_values([0], ascending=[False])
df2 = df2[['name', 0]].rename(columns={'name': 'invited_by_user_profile'})


invite = pd.concat([df1, df2], axis=0)
invite.rename(columns={0:'number_of_invites'},inplace=True)

###

###

fig = px.bar(invite.head(20), y='number_of_invites',
             x='invited_by_user_profile', text='number_of_invites',color_discrete_sequence =['royalblue'])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(
    xaxis_title="Invited by User",
    yaxis_title="Numer of Invites",
    font=dict(
        family="Raleway, monospace",
        size=12,
        color="RebeccaPurple"
    ),
    title={
        'text': "Top 20 Inviters",
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    template='simple_white'
)
fig.show()

###

In [ ]:
print('3214 have no invite linked to their profile')
print('David Castain has invited the most nummber of users')

In [ ]:
#D9. Check if all the information is present for a profile

###

user_data['complete_profile'] = np.where(((user_data['photo_url'] != 0) & (user_data['twitter'] != 0) & (
    user_data['instagram'] != 0) & (user_data['invited_by_user_profile'] != 0)), 'Yes', 'No')

complete_profile_df = user_data.groupby(['follower_status', 'complete_profile']).size(
).reset_index().rename(columns={0: 'count'})

complete_profile_df['%'] = complete_profile_df.groupby(
    ['follower_status'])['count'].transform(lambda x: round(x/np.sum(x)*100))

complete_profile_df = complete_profile_df.sort_values(
    ['count'], ascending=[False])

###

###

fig = px.bar(complete_profile_df, y="follower_status", x="%", color="complete_profile",  orientation='h',
             color_discrete_sequence =['royalblue','cyan'])
fig.update_layout(
    xaxis_title="% Users",
    yaxis_title="Profile with followers",
    font=dict(
        family="Raleway, monospace",
        size=12,
        color="RebeccaPurple"
    ),
    title={
        'text': "Profile completeness",
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    template='simple_white',
   
)

fig.show()

###

In [ ]:
print('For Users with more than 1k followers 56% of the users have a complete profile')
print('While 80% of users with less than 1k followers are missing some information from their profile')

###### D8. Analaysis based on emoji's present in the name

D8. Analaysis based on emoji's present in the name

In [ ]:
# D8.1 Get emoji's present in the name and their description

###

# Get emoji and emoji description from the 'name' column
user_data['emoji_in_name_text'] = extract_emoji(user_data['name'])[
    'emoji_text']

user_data['emoji_in_name_text'] = user_data['emoji_in_name_text'].astype(str)

user_data['emoji_in_name'] = extract_emoji(user_data['name'])['emoji']

user_data['emoji_count_name'] = extract_emoji(user_data['name'])[
    'emoji_counts']

###

D8.2 Number of users with emoji's in their name

In [ ]:
###

# Filter data for user with emoji in  their name
emoji_users = user_data[user_data.astype(str)['emoji_in_name'] != '[]']

print(f'Number of users with emoji in their name : {emoji_users.shape[0]}')

###

D8.3 Get the most commonly used emoji's

In [ ]:
###

# Get most common emoji in username
print('Emoji                        Count')
print(user_data[user_data.astype(str)['emoji_in_name_text'] != '[]']
      ['emoji_in_name_text'].value_counts().head(5).to_string())

###

D8.3 Check if any flags are present in the name

In [ ]:
###

# Check if flag is present in any name
flag_df = pd.DataFrame([row for index, row in emoji_users.iterrows()
                  for key in row['emoji_in_name_text'].split(',') if 'flag:' in key])

flag_df[['name', 'username', 'emoji_in_name_text','num_followers']]

###

D8.4 Check for specific skin tone emojis

In [ ]:
###

#check for skin tone emojis
skin_tone_df = pd.DataFrame([row for index, row in emoji_users.iterrows()
                  for key in row['emoji_in_name_text'].split(',') if 'skin' in key])

skin_tone_df[['name', 'username', 'emoji_in_name_text','num_followers']]

###

D8.5 Check for Rocket emoji in names

In [ ]:
###

# Rocket emoji
rocket_df = pd.DataFrame([row for index, row in emoji_users.iterrows()
                  for key in row['emoji_in_name_text'].split(',') if 'rocket' in key])

rocket_df[['name', 'username', 'num_followers', 'num_following']]

###

D8.6 Check for writing emoji in name

In [ ]:
###

# writing emoji
write_df = pd.DataFrame([row for index, row in emoji_users.iterrows()
                  for key in row['emoji_in_name_text'].split(',') if 'writ' in key])

write_df[['name', 'username', 'num_followers', 'num_following', 'emoji_in_name_text']]

###

D8.9 Check for camera emoji in name

In [ ]:
###

# camera emoji
camera_df = pd.DataFrame([row for index, row in emoji_users.iterrows()
                  for key in row['emoji_in_name_text'].split(',') if 'camera' in key])

camera_df[['name', 'username', 'num_followers', 'num_following']]

###

D8.9 Check for tech emoji in name

In [ ]:
###

# Tech emoji
tech_df = pd.DataFrame([row for index, row in emoji_users.iterrows()
                  for key in row['emoji_in_name_text'].split(',') if 'tech' in key])
tech_df[['name', 'username','emoji_in_name_text', 'num_followers', 'num_following']]

###

D8.10 Check for gender based emojis in name

In [ ]:
###

# Gender based on emoji description
gender_df = pd.DataFrame([row for index, row in emoji_users.iterrows()
                  for key in row['emoji_in_name_text'].split(',') if 'man' in key])
gender_df[['name', 'username','emoji_in_name_text', 'num_followers', 'num_following']]

###

In [ ]:
#--------------------------------------------------------------------------------------------------------------#